# Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -q kaggle

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, LayerNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow import keras
%load_ext tensorboard


import os

# Data Loading

In [4]:
def create_kaggle_dir(kaggle_dir="/root/.kaggle/"):
    if not os.path.exists(kaggle_dir):
        !mkdir $kaggle_dir

def retrieve_kaggle_json(source_dir="/content/drive/MyDrive/", kaggle_dir="/root/.kaggle/", file_name="kaggle.json"):
    source_file = os.path.join(source_dir, file_name)
    target_file = os.path.join(kaggle_dir, file_name)

    if not os.path.exists(target_file):
        !cp $source_file $kaggle_dir
        !chmod 600 $target_file

def download_facemask_dataset():
    if not os.path.exists("/content/face-mask-detection.zip"):
        !kaggle datasets download -d dhruvmak/face-mask-detection

def unzip_facemask_dataset():
    if not os.path.exists("/content/face-mask-detection"):
        !unzip face-mask-detection.zip -d face-mask-detection > /dev/null 2>&1

def dataset_pipeline():
    create_kaggle_dir()
    retrieve_kaggle_json()
    download_facemask_dataset()
    unzip_facemask_dataset()

dataset_pipeline()

 99% 192M/194M [00:07<00:00, 34.5MB/s]
100% 194M/194M [00:07<00:00, 28.8MB/s]


In [5]:
IMG_SIZE = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0, 
                                                          shear_range=0.2, 
                                                          zoom_range=0.2, 
                                                          rotation_range=45, 
                                                          horizontal_flip=True, 
                                                          validation_split=0.2)

In [6]:
train = datagen.flow_from_directory('/content/face-mask-detection/dataset/', target_size=(IMG_SIZE,IMG_SIZE), batch_size=16, 
                                    class_mode='binary', interpolation='lanczos', shuffle=True, subset='training')

test = datagen.flow_from_directory('/content/face-mask-detection/dataset/', target_size=(IMG_SIZE,IMG_SIZE), batch_size=16, 
                                    class_mode='binary', interpolation='lanczos', shuffle=True, subset='validation')

Found 352 images belonging to 2 classes.
Found 88 images belonging to 2 classes.


# Model Creation

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((IMG_SIZE,IMG_SIZE,3)),
    tf.keras.applications.MobileNet(include_top=False, weights='imagenet'),
    tf.keras.layers.Conv2D(3, 3, padding='same', activation='sigmoid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

17225924/17225924 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, None, None, 1024)  3228864  
 nal)                                                            
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 3)           27651     
                                                                 
 flatten (Flatten)           (None, 147)               0         
                                                                 
 dense (Dense)               (None, 1)                 148       
                                                                 
Total params: 3,256,663
Trainable params: 3,234,775
Non-trainable params: 21,888
_________________________________________________________________


In [8]:
model.fit(train, epochs=5, validation_data=test)

Epoch 1/5
22/22 [==============================] - 42s 774ms/step - loss: 0.3075 - accuracy: 0.8580 - val_loss: 0.7861 - val_accuracy: 0.5227
Epoch 2/5
22/22 [==============================] - 16s 747ms/step - loss: 0.1159 - accuracy: 0.9716 - val_loss: 0.1688 - val_accuracy: 0.9659
Epoch 3/5
22/22 [==============================] - 17s 762ms/step - loss: 0.0639 - accuracy: 0.9858 - val_loss: 0.7387 - val_accuracy: 0.6136
Epoch 4/5
22/22 [==============================] - 16s 727ms/step - loss: 0.0319 - accuracy: 0.9972 - val_loss: 0.1478 - val_accuracy: 0.9318
Epoch 5/5
22/22 [==============================] - 16s 744ms/step - loss: 0.0396 - accuracy: 0.9886 - val_loss: 0.0345 - val_accuracy: 1.0000


In [9]:
model.evaluate(test)

6/6 [==============================] - 3s 464ms/step - loss: 0.0379 - accuracy: 1.0000


[0.03790757432579994, 1.0]